## 为了更好的分析数据、首先做如下清理工作

>1. 计算每个订单的骑行持续时间：“duration” 单位为分钟
>2. 检查某些变量在不同的记录中是否有重复
>3. 移除部分变量、排序
>4. 提取年、月、日、时、周

In [1]:
# 导入数据清理所需要的模块

import datetime
import pandas  as pd
from collections import Counter

In [2]:
# 读取数据为 DataFrame 格式
# 预览数据

df = pd.read_csv('mobike_shanghai_sample_updated.csv')

df.info() 
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null object
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null object
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
track               102361 non-null object
dtypes: float64(4), int64(3), object(3)
memory usage: 7.8+ MB


,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1..."


### 1. 计算每个订单的骑行持续时间：“duration” 单位为分钟

In [3]:
# 计算每个订单的骑行持续时间 “duration” 、单位为分钟
# 测试发现直接在 DataFrame 格式下使用 for 循环处理计算耗时太久
# 故对 “duration” 先用列表单独处理、然后转换成 DataFrme 再与原始数据合并

duration_list = []
start_time_only = df.start_time
end_time_only = df.end_time

for i in range(0, len(df.orderid)):
    start = datetime.datetime.strptime(start_time_only[i], "%Y-%m-%d %H:%M")
    end = datetime.datetime.strptime(end_time_only[i], "%Y-%m-%d %H:%M")
    duration_seconds = (end - start).seconds
    duration_minutes = int(duration_seconds / 60)
    duration_list.append(duration_minutes)
    
duration_df = pd.DataFrame(duration_list)
duration_df.columns = ['duration']
df['duration'] = duration_df

In [4]:
# 检查处理后的数据是否符合预期

df.info() 
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 11 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null object
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null object
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
track               102361 non-null object
duration            102361 non-null int64
dtypes: float64(4), int64(4), object(3)
memory usage: 8.6+ MB


,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,duration
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1...",7
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1...",22


### 2. 检查某些变量在不同的记录中是否有重复

In [5]:
# 单车的使用频次记录
# 一共有多少辆不同的单车

bike_duplicated_dict = Counter(df.bikeid)
bike_duplicated_counter = bike_duplicated_dict.values()
bike_unique_counter = len(bike_duplicated_counter)

print ('单车的总量：%d' % (bike_unique_counter))

单车的总量：79062


In [6]:
# 用户的下单频次记录
# 一共有多少用户

user_duplicated_dict = Counter(df.userid)
user_duplicated_counter = user_duplicated_dict.values()
user_unique_counter = len(user_duplicated_counter)

print ('用户的总数：%d' % (user_unique_counter))

用户的总数：16887


### 3. 移除部分变量、排序

In [7]:
# 由于轨迹点和骑行顺序不一定一致故移除 track
# 按照订单顺序对数据排序

df.drop(['track'], axis=1, inplace=True)

df = df.sort_values(by=['orderid'])
df.to_csv('mobike_data_cleared.csv', index=False)
df = pd.read_csv('mobike_data_cleared.csv')

In [8]:
# 检查处理后的数据是否符合预期

df.info() 
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null object
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null object
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
duration            102361 non-null int64
dtypes: float64(4), int64(4), object(2)
memory usage: 7.8+ MB


,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,duration
0,6,288841,4427,2016-08-28 00:00,121.462,31.313,2016-08-28 00:17,121.455,31.315,17
1,24,53494,17056,2016-08-28 00:00,121.483,31.262,2016-08-28 00:16,121.479,31.285,16


### 4. 提取年、月、日、时、周

In [9]:
# 从订单开始时间中提取年、月、日、时、周

year_list = []
month_list = []
day_list = []
hour_list = []
weekday_list = []
time = df.start_time

for i in range(0, len(time)):
    strptime = datetime.datetime.strptime(time[i], "%Y-%m-%d %H:%M")
    
    year_list.append(strptime.year)
    month_list.append(strptime.month)
    day_list.append(strptime.day)
    hour_list.append(strptime.hour)
    weekday_list.append(strptime.weekday() + 1)
    
year_df = pd.DataFrame(year_list)
month_df = pd.DataFrame(month_list)
day_df = pd.DataFrame(day_list)
hour_df = pd.DataFrame(hour_list)
weekday_df = pd.DataFrame(weekday_list)

year_df.columns = ['year']
month_df.columns = ['month']
day_df.columns = ['day']
hour_df.columns = ['hour']
weekday_df.columns = ['weekday'] 

df['year'] = year_df
df['month'] = month_df
df['day'] = day_df
df['hour'] = hour_df
df['weekday'] = weekday_df

In [10]:
# 检查处理后的数据是否符合预期

df.info() 
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 15 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null object
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null object
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
duration            102361 non-null int64
year                102361 non-null int64
month               102361 non-null int64
day                 102361 non-null int64
hour                102361 non-null int64
weekday             102361 non-null int64
dtypes: float64(4), int64(9), object(2)
memory usage: 11.7+ MB


,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,duration,year,month,day,hour,weekday
0,6,288841,4427,2016-08-28 00:00,121.462,31.313,2016-08-28 00:17,121.455,31.315,17,2016,8,28,0,7
1,24,53494,17056,2016-08-28 00:00,121.483,31.262,2016-08-28 00:16,121.479,31.285,16,2016,8,28,0,7


In [11]:
# 暂存清理过的数据

df.to_csv('mobike_data_cleared.csv', index=False)

In [12]:
# 查看年、月、日、时间、周、单次骑行时间的分布

df.year.describe(), df.month.describe(), df.day.describe(), df.hour.describe(), df.weekday.describe(), df.duration.describe()


(count    102361.0
 mean       2016.0
 std           0.0
 min        2016.0
 25%        2016.0
 50%        2016.0
 75%        2016.0
 max        2016.0
 Name: year, dtype: float64, count    102361.0
 mean          8.0
 std           0.0
 min           8.0
 25%           8.0
 50%           8.0
 75%           8.0
 max           8.0
 Name: month, dtype: float64, count    102361.000000
 mean         18.839363
 std           8.514200
 min           1.000000
 25%          12.000000
 50%          20.000000
 75%          26.000000
 max          31.000000
 Name: day, dtype: float64, count    102361.000000
 mean         14.489190
 std           5.302643
 min           0.000000
 25%          10.000000
 50%          16.000000
 75%          19.000000
 max          23.000000
 Name: hour, dtype: float64, count    102361.000000
 mean          3.794502
 std           1.996931
 min           1.000000
 25%           2.000000
 50%           4.000000
 75%           6.000000
 max           7.000000
 Name: w